In [ ]:
import json
import pyspark as ps
from pyspark.sql.types import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
%matplotlib inline

plt.rcParams['figure.figsize'] = [30, 15]
